## Install packages
Run this only once

In [ ]:
# Install packages
conda install tensorflow
conda install keras
conda install -c conda-forge shapely

## Import files and load dataset

In [5]:
# Import files
import numpy as np
import pandas as pd
import math
from typing import Any, List, Tuple, Set, Dict, Sequence, Iterator, Iterable, Generator
import requests, gzip, os, hashlib
%pylab inline
import doctest
import seaborn as sns
import csv

from keras.datasets import mnist
from matplotlib import pyplot

import matplotlib.pyplot as plt
%matplotlib inline

import random
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

import itertools

Populating the interactive namespace from numpy and matplotlib


C:\Users\20183524\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random', 'Generator']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [6]:
#loading the dataset
(train_X, train_y), (test_X, test_y) = mnist.load_data()

## Standard functions

In [7]:
def plot_digit(image):
    "Plots a digit"
    fig = plt.figure
    plt.imshow(image, cmap='gray')
    plt.show()

#### Weighted covariance
Returns the weighted covariance matrix

`data`: list of arrays, containing
- horizontal position
- vertical position
- grayscale value

of each pixel

In [8]:
def weighted_Cov(image: Iterator[Iterator[int]], pixel_mids: Iterator[Iterator[float]], d: int = 28):
    """Returns the weighted covariance matrix for a digit I.
    image: list of lists containing graycale values
    pixel_mids: coordinates of pixel's middle points, ordered as in image
    d: image dimensions (standard 28)
    """
    
    # Compute sum of grayscale values
    V_1 = sum([sum(pixels) for pixels in image])
    
    # Weighted averages X and Y
    X_bar, Y_bar = 0, 0
    for x in range(d):
        for y in range(d):
            X_bar += image[x][y] * pixel_mids[x][y][0]
            Y_bar += image[x][y] * pixel_mids[x][y][1]
    X_bar = X_bar / V_1
    Y_bar = Y_bar / V_1
    
    # Weighted variances
    var_X = 0
    var_Y = 0
    cov_XY = 0
    for x in range(d):
        for y in range(d):
            var_X += image[x][y] * ((pixel_mids[x][y][0] - X_bar)**2)
            var_Y += image[x][y] * ((pixel_mids[x][y][1] - Y_bar)**2)
            cov_XY += image[x][y] * (pixel_mids[x][y][0] - X_bar) * (pixel_mids[x][y][1] - Y_bar)
    var_X = var_X / V_1
    var_Y = var_Y / V_1
    cov_XY = cov_XY / V_1
    
    return array([[var_X, cov_XY], [cov_XY, var_Y]])

#### Principal axis

In [10]:
def principal_axis(cov_Matrix: Iterator[Iterator[float]]):
    """Returns the principal axis (ordered by new horizontal, vertical).
    cov_Matrix: covariance matrix of an image
    """
    
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(cov_Matrix)
    
    # Set principal axis to be the second in the list of axes
    if abs(eigenvectors[0][1]) > abs(eigenvectors[1][1]):
        eigenvectors = [ eigenvectors[1], eigenvectors[0] ]
        eigenvalues = [ eigenvalues[1], eigenvalues[0] ] 

    # Ensure proper directions of eigenvectors
    if eigenvectors[0][0] < 0: # Secondary axis is in the second or third quadrant
        eigenvectors[0] = [ -eigenvectors[0][0], -eigenvectors[0][1] ]
    if eigenvectors[1][1] < 0: # Primary axis is in the third or fourth quadrant
        eigenvectors[1] = [ -eigenvectors[1][0], -eigenvectors[1][1] ]
      
    return eigenvectors

#### t functions

In [11]:
def find_z_c (axes: Iterator[Iterator[float]], d: int = 28):
    """Returns transformation parameters z and c
    axes: projection matrix, i.e. unit vectors of the axes
    d: dimension (nr of pixels)
    """
    if axes[1][0] > 0 and axes[1][1] > 0: # Principal axis in first quadrant
        c = axes[1][0] * d + axes[1][1] * d
        z = [ axes[0][1] * d, 0 ]
    else: # Principal axis in second quadrant
        c = axes[0][0] * d + axes[0][1] * d
        z = [ 0, axes[1][0] * d ]
    
    return z, c / d

def inverse_2x2 (A):
    """Returns the inverse of the 2x2 matrix A
    Assumptions: det(A) != 0, i.e. A[0][0] * A[1][1] - A[0][1] * A[1][0] != 0
    """
    
    det = A[0][0] * A[1][1] - A[0][1] * A[1][0]
    
    return [ [ A[1][1]/det, -A[0][1]/det ], [ -A[1][0]/det, A[0][0]/det ] ]

def function_t (A, z, c, p):
    """Computes the transformation map of p (to the new grid)
    A: projection matrix (i.e. unit vectors of axes)
    z: vector for positivity
    c: scaling factor
    """
    
    x = ((A[0][0] * p[0]) + (A[0][1] * p[1])) - z[0]
    y = (A[1][0] * p[0] + A[1][1] * p[1]) - z[1]
    
    return [ x/c, y/c ]

def inverse_t (A, z, c, p):
    """Computes the inverse transformation of p (to the original grid)
    A: projection matrix (i.e. unit vectors of axes)
    z: vector for positivity
    c: scaling factor
    """
    
    A_inv = inverse_2x2(A)
    
    x = (A_inv[0][0] * (c * p[0] + z[0]) + A_inv[0][1] * (c * p[1] + z[1]))
    y = (A_inv[1][0] * (c * p[0] + z[0]) + A_inv[1][1] * (c * p[1] + z[1]))
    
    return [ x, y ]

#### Find grayscale value

In [12]:
def find_g (image, A, z, c, p, d = 28):
    """Finds the grayscale value of a point p in the new grid
    image: list of rows with pixels
    A: projection matrix (i.e. unit vectors of axes)
    z: vector for positivity
    c: scaling factor
    p: point at which grayscale value needs to be determined
    d: dimension of image (standard 28)
    """
    
    # Find original coordinates
    q = inverse_t(A, z, c, p)
    
    # Determine position of coordinates in image
    column = math.floor(q[0])
    row = d - math.ceil(q[1])
    
    if 0 <= row <= d - 1 and 0 <= column <= d - 1: # Pixel can be transformed to old grid
        return image[ row ][ column ]
    
    return 0

#### Rasterize

In [13]:
def rasterize (image, A, z, c, d = 28):
    """ Projects an image onto axes A and forms a new image of the results.
    image: original list with rows of grayscale values
    A: projection axes
    z: transformation vector
    c: transformation scale
    d: dimensions of image (standard 28)
    """
    # Initialize list of grayscale values
    grayscale = []
    m = 5 # Division of pixel into m x m subpixels
    
    for y in range(d):
        y = d - y - 1 # Actual coordinate value
        
        # Initialize list of grayscale values in a row
        row = []
        for x in range(d):
            
            g = 0

            for k in range(0, m):
                x_sub = x + k/m + 0.5 * (1/m) # x-value of subpixel

                for l in range(0, m, 1):
                    y_sub = y + l/m + 0.5 * (1/m) # y-value of subpixel
                    # Add grayscale value of subpixel
                    g += find_g(image, A, z, c, [x_sub, y_sub])
            row.append(int(round(g / (m**2))))
            
        grayscale.append(row)
    
    return grayscale

#### Perform PCA
Executes the PCA procedure for an image

In [14]:
d = 28
pixel_mids_arranged = [ [ ((x + 0.5) % d, d - y - 0.5) for x in range(d)] for y in range(d) ]

def perform_PCA (image, d = 28):
    """Executes PCA process for image.
    image: list of rows with grayscale values
    """
    
    # Compute covariance matrix and determine axes
    covariance_matrix = weighted_Cov(image, pixel_mids_arranged)
    axes = principal_axis(covariance_matrix)
    
    # Find constants for transformation map
    z, c = find_z_c(axes)

    # Produce new image by projection onto axes
    return rasterize(image, axes, z, c)

# Execute PCA for all digits

In [ ]:
train_X_PCA = []
for i in range(len(train_X)):
    train_X_PCA.append( perform_PCA( train_X[i] ) )